In [1]:
import pandas as pd
import numpy as np

In [2]:
boolean_matrix = pd.read_csv('tf.csv')
boolean_matrix.head()

,name,1818,taking,epistolary,forgiv,lov,hum,what,then,august,...,growing,wal,goes,winds,releaseslistsexplorenews,comt,conferenc,com,months,project
0,http://quotes.toscrape.com/,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,http://quotes.toscrape.com/tag/inspirational/,0,0,0,0,1,0,1,1,0,...,0,0,1,0,0,0,0,1,0,0
2,http://quotes.toscrape.com/tag/paraphrased/pag...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,http://quotes.toscrape.com/tag/obvious/page/1/,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,http://quotes.toscrape.com/tag/misattributed-e...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# get_pj_qj

In [3]:
def get_pj_qj(boolean_matrix: pd.DataFrame)-> tuple:
    row_size = boolean_matrix.shape[1]
    pj = np.full((row_size), 0.5)
    qj = np.array(boolean_matrix.sum(axis=0) / row_size)
    return pj, qj

In [4]:
values = boolean_matrix.iloc[:,1:] # tomar solo las columnas con numero, ignorar la url
pj, qj = get_pj_qj(values)
print(pj)
print(qj)

[0.5 0.5 0.5 ... 0.5 0.5 0.5]
[0.00054795 0.00054795 0.00054795 ... 0.00438356 0.00054795 0.00054795]


# Consulta

In [5]:
import process_text
import re

In [6]:
consulta_raw = '1818 Robert GalbraithAlthough she writes under the pen name J.K. Rowling, pronounced like rolling, her name when her first Harry Potter book was published was simply Joanne Rowling. Anticipating that the target audience of young boys might not want to read a book written by a woman, her publishers demanded that she use two initials, rather than her full name. As she had no middle name, she chose K as the second initial of her pen name, from her paternal grandmother Kathleen Ada Bulgen Rowling. She calls herself Jo and has said, "No one ever called me \'Joanne\' when I was young, unless they were angry." Following her marriage, she has sometimes used the name Joanne Murray when conducting personal business."'

In [7]:
corpus_stems = boolean_matrix.columns[1:]

In [27]:
def vector_consulta(consulta, corpus_stems):
    tokens = re.split(r"[^a-z0-9]+", consulta)
    query_data = process_text.ProcessData('query', tokens)
    vector_query = []
    for stem in corpus_stems:
        if stem in query_data.stems:
            vector_query.append(1)
        else:
            vector_query.append(0)
    return np.array(vector_query)

In [28]:
q = vector_consulta(consulta_raw, corpus_stems)

# semejanza

In [30]:
def get_sem_table(consulta_vector, boolean_matrix):
    q = consulta_vector
    pj, qj = get_pj_qj(boolean_matrix)
    semejanza = ( (q * boolean_matrix) *  np.log10((  pj*(1-qj)  )/ (  qj*(1-pj)  )) ).sum(axis=1)
    return semejanza

In [31]:
semejanza = get_sem_table(q, np.array(values))
semejanza

array([ 10.52556732,  17.3232277 ,   1.79190833,   1.89402695,
         4.25029189,   1.89402695,  26.82770647,  19.63397502,
         1.79190833,   1.79190833,   2.48168145,  26.82770647,
         2.48168145,  14.02818327,   1.89402695,   9.54278274,
         0.        ,  10.39390009,   4.27358978,   9.89135707,
        17.89573345,  24.81223417,   2.17921656,  20.45683483,
        10.0907124 ,   8.00936668,   2.48168145,   2.00168565,
         8.00936668,  20.45683483,   4.71936917,   0.        ,
         0.        ,  17.3232277 ,   0.        ,  10.0907124 ,
        21.04071298,   3.79359398,   0.        ,   3.79359398,
         0.        ,   0.        ,   1.79190833,   0.        ,
         0.        ,  23.1313863 ,  10.52556732, 137.92603913,
        15.75478851,  22.63924799])

# Concatenar nombre y ordenar

In [22]:
resultado = boolean_matrix[['name']].copy()
resultado['semejanza'] = semejanza
resultado.head()

,name,semejanza
0,http://quotes.toscrape.com/,10.525567
1,http://quotes.toscrape.com/tag/inspirational/,17.323228
2,http://quotes.toscrape.com/tag/paraphrased/pag...,1.791908
3,http://quotes.toscrape.com/tag/obvious/page/1/,1.894027
4,http://quotes.toscrape.com/tag/misattributed-e...,4.250292


In [26]:
ordenados = resultado.sort_values(by=['semejanza'], ascending=False)
ordenados.head()

,name,semejanza
47,http://quotes.toscrape.com/author/J-K-Rowling/,137.926039
11,http://quotes.toscrape.com/tag/love/page/1/,26.827706
6,http://quotes.toscrape.com/tag/love/,26.827706
21,https://www.zyte.com,24.812234
45,http://quotes.toscrape.com/author/Jane-Austen/,23.131386


# get_result(query, matriz_booleana) -> lista ordenada  

In [ ]:
def get_result(query_raw: str, matrix_boolean: pd.DataFrame) -> list:
    #### hacer vector query
    #### calcular 

# PRUEBAS

In [103]:
# Define two NumPy arrays
array1 = np.array([0.5, 0.5, 0.5, 0.5, 0.5])
array2 = np.array([2, 3, 4, 5, 6])

# Perform element-wise multiplication
result = array1 * array2

# The 'result' array will contain the element-wise multiplication
print(result)


[1.  1.5 2.  2.5 3. ]


In [112]:
import numpy as np

# Define two matrices, each with rows representing vectors
q = np.array([1, 0, 1])
tabla = np.array([[1, 0, 1], [1, 1, 1], [0, 1, 1]])

# Perform element-wise multiplication between the two matrices
tabla_X_q = q * tabla

# The 'result' matrix will contain the element-wise multiplication
print(tabla_X_q)

[[1 0 1]
 [1 0 1]
 [0 0 1]]


In [136]:
pj = np.array([0.5, 0.5, 0.5])
qj = np.array([0.4, 0.5, 0.3])

In [137]:
1-qj

array([0.6, 0.5, 0.7])

In [138]:
pj*(1-qj)

array([0.3 , 0.25, 0.35])

In [139]:
(1-pj)

array([0.5, 0.5, 0.5])

In [140]:
qj*(1-pj)

array([0.2 , 0.25, 0.15])

In [141]:
(  pj*(1-qj)  )/ (  qj*(1-pj)  )

array([1.5       , 1.        , 2.33333333])

In [145]:
log_pj_qj_operation = np.log10((  pj*(1-qj)  )/ (  qj*(1-pj)  ))
log_pj_qj_operation

array([0.17609126, 0.        , 0.36797679])

In [152]:
tabla_bayes = tabla_X_q * log_pj_qj_operation
tabla_bayes

array([[0.17609126, 0.        , 0.36797679],
       [0.17609126, 0.        , 0.36797679],
       [0.        , 0.        , 0.36797679]])

In [155]:
semejanza = tabla_bayes.sum(axis=1)
semejanza

array([0.54406804, 0.54406804, 0.36797679])

In [33]:
( (q * boolean_matrix) *  np.log10((  pj*(1-qj)  )/ (  qj*(1-pj)  )) )

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/erick/tmp/crawling/roboto/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_235107/4066473308.py", line 1, in <module>
    ( (q * boolean_matrix) *  np.log10((  pj*(1-qj)  )/ (  qj*(1-pj)  )) )
       ~~^~~~~~~~~~~~~~~~
  File "/home/erick/tmp/crawling/roboto/venv/lib/python3.11/site-packages/pandas/core/generic.py", line 2102, in __array_ufunc__
    return arraylike.array_ufunc(self, ufunc, method, *inputs, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/erick/tmp/crawling/roboto/venv/lib/python3.11/site-packages/pandas/core/arraylike.py", line 273, in array_ufunc
    result = maybe_dispatch_ufunc_to_dunder_op(self, ufunc, method, *inputs, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "ops_dispatch.pyx", line 113, 